In [1]:
#Import required library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras.utils import to_categorical
from keras.models import load_model

In [2]:
#Read Dataset
#Preprocessing
df = pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
#dropping unwanted columns
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [4]:
# Count of Spam and Ham values

df.groupby(['v1']).size()

v1
ham     4825
spam     747
dtype: int64

In [5]:
# Label Encoding target column

X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)
# Test and train split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)
#Tokenisation function
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)

sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [6]:
inputs = Input(name='InputLayer',shape=[max_len])  #Create Model and Add Layers (LSTM)
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

In [7]:
model = Model(inputs=inputs,outputs=layer)   #COMPILE THE MODEL
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FullyConnectedLayer1 (Dense  (None, 256)              16640     
 )                                                               
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                             

In [8]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,  #FIT THE MODEL
          validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 11s 270ms/step - loss: 0.3156 - accuracy: 0.8820 - val_loss: 0.1460 - val_accuracy: 0.9726
Epoch 2/10
30/30 [==============================] - 8s 252ms/step - loss: 0.0801 - accuracy: 0.9805 - val_loss: 0.0736 - val_accuracy: 0.9715
Epoch 3/10
30/30 [==============================] - 9s 288ms/step - loss: 0.0399 - accuracy: 0.9876 - val_loss: 0.0705 - val_accuracy: 0.9789
Epoch 4/10
30/30 [==============================] - 7s 249ms/step - loss: 0.0288 - accuracy: 0.9921 - val_loss: 0.0647 - val_accuracy: 0.9789
Epoch 5/10
30/30 [==============================] - 7s 239ms/step - loss: 0.0228 - accuracy: 0.9945 - val_loss: 0.0670 - val_accuracy: 0.9768
Epoch 6/10
30/30 [==============================] - 7s 240ms/step - loss: 0.0162 - accuracy: 0.9955 - val_loss: 0.0885 - val_accuracy: 0.9726
Epoch 7/10
30/30 [==============================] - 11s 386ms/step - loss: 0.0128 - accuracy: 0.9960 - val_loss: 0.0802 - val_accuracy: 0.9789
Epoc

In [15]:
model.save("model1")   #SAVE THE MODEL

In [11]:
test_sequences = tok.texts_to_sequences(X_test) #TEST THE MODEL
test_sequences_matrix  = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [12]:
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.3f}'.format(accuracy[1]))

27/27 [==============================] - 1s 23ms/step - loss: 0.1001 - accuracy: 0.9856
Accuracy: 0.986


In [13]:
y_pred = model.predict(test_sequences_matrix)
print(y_pred[25:40].round(3))

27/27 [==============================] - 1s 23ms/step
[[0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.997]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [1.   ]]


In [14]:
print(Y_test[25:40])

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]]
